# Практическое задание к уроку 3 часть 2

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import numpy as np


In [2]:
print(tf.__version__)
msg = tf.constant('TensorFlow 2.0 Hello World')
tf.print(msg)

2.0.0
TensorFlow 2.0 Hello World


In [14]:
builder = tfds.builder('imdb_reviews')
builder.download_and_prepare()

# Default parameters: Returns the dict of tf.data.Dataset
ds_all_dict = builder.as_dataset()
assert isinstance(ds_all_dict, dict)
print(ds_all_dict.keys())  # ==> ['test', 'train', 'unsupervised']

assert isinstance(ds_all_dict['test'], tf.data.Dataset)
ds_all_dict

dict_keys(['test', 'train', 'unsupervised'])


{'test': <PrefetchDataset shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>,
 'train': <PrefetchDataset shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>,
 'unsupervised': <PrefetchDataset shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>}

In [3]:
ds = tfds.load('imdb_reviews', split='train', shuffle_files=True)

In [28]:
ds_test = tfds.load('imdb_reviews', split='test', shuffle_files=True)

In [29]:
for example in ds.take(2):
    text, label = example["text"], example["label"]
    print(text, label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on t

In [30]:
tokenizer = tfds.features.text.Tokenizer()

In [31]:
vocabulary_set = set()
for  text_tensor in ds:
    some_tokens = tokenizer.tokenize(text_tensor['text'].numpy())
    vocabulary_set.update(some_tokens)
    vocab_size = len(vocabulary_set)
vocab_size

93246

In [32]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [33]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

def encode_map_fn(x ):
  label=['label']
  text=x['text']
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([120])
  label.set_shape([])
  return label, encoded_text


#all_encoded_data = ds_all_dict['train'].map(encode_map_fn)

In [9]:
def vectorize_text(tf_text):
    #text = tf.expand_dims(text, -1)
    encoded_text, label = tf.py_function(encode, 
                                       inp=[tf_text['text'], tf_text['label']], 
                                       Tout=(tf.int64, tf.int64))
    print (tf_text['text'],label)
    encoded_text.set_shape([None])
    label.set_shape([])
    return encoded_text,label

train_ds = ds.map(lambda x :vectorize_text(x))


Tensor("args_1:0", shape=(), dtype=string) Tensor("EagerPyFunc:1", dtype=int64)


In [10]:
train_ds

<MapDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>

In [11]:

for example in train_ds.take(2):
    #text, label = example["text"], example["label"]
    print(example)
  

(<tf.Tensor: id=50099, shape=(121,), dtype=int64, numpy=
array([ 6913, 21602, 27999, 28075, 57905, 92944, 81037,  3358, 46753,
       69251, 47861,  3993, 16122, 77287, 25328, 45522, 73672, 73019,
       73337, 70258,  8326, 56881, 31712,  3218, 61737, 46753,  6986,
       48876,  9469, 47861, 16793, 25466,  6986, 70258, 46179, 48612,
       54560, 57786, 31712, 92944, 65314, 46330, 22870,  6913, 92944,
       89223, 27999, 18018, 36766, 35882, 46028, 31070, 58189, 58630,
       11722,  8236, 17618,  5198, 73802,   559, 62582, 93665, 17618,
       52194,  6986, 42457, 38879, 22085, 70812, 71848, 27763, 91241,
        6716, 45439, 37612, 38176,  7000, 35595, 47964, 77287, 21602,
       14747, 56881, 54251, 11722, 75544, 36528, 47861, 54251, 92944,
       33342, 21602, 20699, 62571, 83977, 71516, 47176, 75097, 92635,
       87611, 33342, 47482, 73337, 45534, 48778, 31712, 22498,  1685,
       65314, 48778, 16122, 77287, 65314,  9587, 75620, 75097, 48612,
       33400, 56594, 62710,  7046

In [12]:
vocab_size += 1

In [13]:
model = keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
  #  tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

In [14]:
model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [15]:
model.fit(train_ds,epochs=2)

Epoch 1/2
25000/25000 [==============================] - 3887s 155ms/step - loss: 7.7125 - accuracy: 0.4947
Epoch 2/2
25000/25000 [==============================] - 4254s 170ms/step - loss: 7.7122 - accuracy: 0.4947


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          6011520   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 6,085,889
Trainable params: 6,085,889
Non-trainable params: 0
_________________________________________________________________


In [34]:
test_ds = ds_test.map(lambda x :vectorize_text(x))


Tensor("args_1:0", shape=(), dtype=string) Tensor("EagerPyFunc:1", dtype=int64)


In [35]:
test_loss, test_acc = model.evaluate(test_ds)


25000/25000 [==============================] - 765s 31ms/step - loss: 7.7125 - accuracy: 0.4997


# Вывод
### увеличить количество эпох на данный момент 2 явно мало. 
### Проварьировать количество нейронов 